In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Pick up from our scraped data
data = json.load(open('scraped_players.json'))

In [3]:
# Here's an example row. Look at the stats property... it's a nested dictionary
data[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [4]:
# We need to recreate tables from our data
# We will create dataframes for each permutation of the data
# So we currently have nested data like this:
#
# player (name, position, etc):
#    -> stats
#        -> career_stats
#            -> stats type (hitting / pitching)
#                -> stats for year 1
#                -> stats for year 2
#                -> ...

# We are going to denormalize "grouped" relationship by unrolling them and repeating each level of data
# So our example above becomes
# name, hitting, year 1, stats...
# name, hitting, year 2, stats...
# name, pitching, year 1, stats...
# name, pitching, year 2, stats...
# etc

In [5]:
# We're going to create a list of stats
data_for_df = []

# We loop over each player in the data we loaded
for player in data:
    # Now we loop over each value in the career_stats items.
    # Not all players have stats so we use `.get` to "defensively" try to get the items
    # So we're saying:
    # give me the stats if it exists, otherwise an empty dictionary
    # Then using that result give me the career_stats key or and empty dictionary
    # and finally the items for the dictionary
    # Note that we can only chain .get and .items because those are attributes on dictionaries
    # and we're defaulting to dictionaries if the item is not found
    for key, val in player.get('stats', {}).get('career_stats', {}).items():
        # Stat type is going from 'Hitting Statistics'
        # to 'hitting statistics'
        # to ['hitting', 'statistics']
        # to 'hitting'
        stat_type = key.lower().split()[0]
        # Players have to have a name and number
        name = player.get('Full Name')
        if not name:
            continue
        num = player.get('#')
        if not num:
            continue
        # Now loop through each value in the career_stats item
        for stat in val:
            # Add the name
            stat['name'] = name
            # Add the player number
            stat['num'] = num
            # Add the stat_type
            stat['stat_type'] = stat_type
            # Append the denormalized row
            # This is now something like
            # {'name': 'John Doe', 'num': '1', 'stat_type': 'hitting', 'year': '2016', ...}
            data_for_df.append(stat)

In [6]:
# We feed our list of dictionaries into the dataframe class to instantiate a new dataframe
df = pd.DataFrame(data_for_df)

In [7]:
# Attempt to convert everything to a numeric value and by specifying
# errors='ignore' the original value will remain in place if it's not numeric
# Note that this returns a new dataframe so we have to assign it back to our df variable
df = df.apply(pd.to_numeric, errors='ignore')

In [8]:
# Drop the avg stat since it's not always present
# Note this time we can use inplace=True and it does NOT return a new dataframe
df.drop('avg', axis=1, inplace=True)

In [9]:
# Generate the batting average and era for all records
# Note again how we have single arithmetic operators
# but our data is a list- this, again, is the power and short cut of pandas / numpy
df['ba'] = df.h / df.ab
df['era2'] = df.r / df.ip

In [10]:
# Now We can generate groups
# We can make a compound group by name and stat type
by_player = df.groupby(('name', 'stat_type'))
# and by year
by_year = df.groupby('year')
# and by type
by_type = df.groupby('stat_type')

In [11]:
# We can also group by a property after fetching a group
# Here's the batting average for the whole team by year
by_type.get_group('hitting').groupby('year').mean().ba

year
2015    0.215686
2016    0.219881
2017    0.241031
Name: ba, dtype: float64

In [12]:
# Here are all the batting averages across the player / type compound group
by_player.mean().h / by_player.mean().ab

name                  stat_type
Ashton McGee          hitting      0.327354
Austin Bergner        hitting           NaN
                      pitching          NaN
Bo Weiss              hitting           NaN
                      pitching          NaN
Brandon Martorano     hitting      0.068182
Brandon Riley         hitting      0.306220
Brendan Illies        hitting      0.210526
Brett Daniels         hitting           NaN
                      pitching          NaN
Cody Roberts          hitting      0.262295
                      pitching          NaN
Gianluca Dalatri      hitting           NaN
                      pitching          NaN
Hansen Butler         hitting           NaN
                      pitching          NaN
Ike Freeman           hitting      0.173913
Jason Morgan          hitting           NaN
                      pitching          NaN
Josh Hiatt            hitting           NaN
                      pitching          NaN
Josh Ladowski         hitting      0.269231


In [13]:
# We can mask out the rows where the era2 and ba is null
# and use the .loc method to specify a subset of columns to view
df[df.era2.isnull() & df.ba.isnull()].loc[:, ('name', 'year', 'era', 'ba')]

,name,year,era,ba
11,Cody Roberts,2017,0.0,NaN
20,Brett Daniels,2015,NaN,NaN
21,Brett Daniels,2016,NaN,NaN
22,Brett Daniels,2017,NaN,NaN
24,Tyler Baum,2017,NaN,NaN
27,Hansen Butler,2015,NaN,NaN
28,Hansen Butler,2016,NaN,NaN
31,Taylor Sugg,2016,NaN,NaN
32,Taylor Sugg,2017,NaN,NaN
36,Jason Morgan,2015,NaN,NaN


In [14]:
# We can compare that with all the batting averages
df.loc[:, ('name', 'year', 'ba')]

,name,year,ba
0,Brandon Riley,2016,0.293814
1,Brandon Riley,2017,0.316964
2,Kyle Datres,2016,0.250000
3,Kyle Datres,2017,0.265217
4,Brandon Martorano,2017,0.068182
5,Ashton McGee,2017,0.327354
6,Ike Freeman,2017,0.173913
7,Zack Gahagan,2015,0.215686
8,Zack Gahagan,2016,0.297297
9,Zack Gahagan,2017,0.242857


In [15]:
# Or all the eras
df.loc[:, ('name', 'year', 'era2')]

,name,year,era2
0,Brandon Riley,2016,NaN
1,Brandon Riley,2017,NaN
2,Kyle Datres,2016,NaN
3,Kyle Datres,2017,NaN
4,Brandon Martorano,2017,NaN
5,Ashton McGee,2017,NaN
6,Ike Freeman,2017,NaN
7,Zack Gahagan,2015,NaN
8,Zack Gahagan,2016,NaN
9,Zack Gahagan,2017,NaN
